## Importing the important libraries

In [44]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
colors = ['#235E72']

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

In [45]:
df = pd.read_csv('data/data.csv')
df.head()

,Movie Name,IMDb Rating,Popularity,Release Date,Box Office,Runtime,Genre,Director,Cast,Production Company,Country,Language
0,The Kerala Story Bangla,6.5,NaN,NaN,NaN,12 minutes,"['Short', 'Drama']",['Satyajit Das'],[],[],NaN,['Bengali']
1,Generation Z,NaN,NaN,NaN,NaN,NaN,"['Horror', 'Comedy']",[],"['Chris Reilly', 'Ellora Torchia', 'Viola Pret...","['All3Media International', 'The Forge', 'Zwei...",NaN,[]
2,Criminal Justice: Adhura Sach,7.7,NaN,"August 26, 2022 (India)",NaN,45 minutes,"['Crime', 'Drama', 'Mystery']",[],"['Pankaj Tripathi', 'Swastika Mukherjee', 'Pur...","['Applause Entertainment Ltd.', 'BBC Studios']",India,['Hindi']
3,Madhil Mel Kaadhal,NaN,NaN,NaN,NaN,NaN,['Romance'],['Anjana Ali Khan'],"['Mugen Rao', 'Divya Bharathi', 'Sakshi Agarwal']",[],NaN,['Tamil']
4,Saawariya,5.2,NaN,"November 9, 2007 (India)","$18,525,631",2 hours 22 minutes,"['Drama', 'Musical', 'Romance']",['Sanjay Leela Bhansali'],"['Ranbir Kapoor', 'Sonam Kapoor', 'Salman Khan']","['SPE Films', 'SLB Films Pvt. Ltd.']",NaN,['Hindi']


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10004 entries, 0 to 10003
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Movie Name          9920 non-null   object 
 1   IMDb Rating         8671 non-null   float64
 2   Popularity          187 non-null    object 
 3   Release Date        6249 non-null   object 
 4   Box Office          2570 non-null   object 
 5   Runtime             7699 non-null   object 
 6   Genre               10004 non-null  object 
 7   Director            10004 non-null  object 
 8   Cast                10004 non-null  object 
 9   Production Company  10004 non-null  object 
 10  Country             5998 non-null   object 
 11  Language            10004 non-null  object 
dtypes: float64(1), object(11)
memory usage: 938.0+ KB


## Data Cleaning

In [47]:
# Droping the columns that have many null values or are not relevant to the analysis
df = df.drop(['Movie Name', 'Popularity', 'Production Company', 'Box Office', 'Country'], axis=1)

In [48]:
for i in range(len(df)):
    j = df.loc[i, 'Language']
    j = j[1:-1].split(',')
    flg = False
    for k in j:
        if k.strip()=="'Hindi'":
            df.loc[i, "isHindi"] = 1
            flg = True
            break
    if not flg:
        df.loc[i, "isHindi"] = 0
df.drop(['Language'], axis=1, inplace=True)

In [49]:
#Drop rows with missing values in IMDB rating
df = df.dropna(subset=['IMDb Rating'])

In [50]:
### Extracting the year of release column from date of release and filling the missing values with mode
df['Release Date'].fillna('July 2023', inplace=True)
years = df['Release Date'].apply(lambda x: (x.split('(')[0])).apply(lambda x: x.split()[-1] if len(x.split(','))==1 else x.split(',')[1]) 
df['Year'] = years
df['Year'] = df['Year'].astype(int)

In [51]:
df.drop('Release Date', axis=1, inplace=True)

In [52]:
# Replacing cast and director columns with the first entry in the list
df['Cast'] = df['Cast'].apply(lambda x: x[1:-1].split(', ')[0] if x != '[]' else np.NaN)
df.dropna(subset=['Cast'], inplace=True)

df['Director'] = df['Director'].apply(lambda x: x[1:-1].split(',')[0] if x!='[]' else np.NaN)
df.dropna(subset=['Director'], inplace=True)

In [53]:
## Cleaning the runtime column by replacing it with number of minutes
df['Runtime'] = df['Runtime'].fillna('120 minutes')
df['Runtime'] = df['Runtime'].apply(lambda x: int(x.split()[0]) if 'hours' not in x else (int(x.split()[0])*60 + int(x.split()[2]) if len(x.split())>2 else int(x.split()[0])*60))

In [54]:
## Opening the Genre column and filling the missing values with mode
df['Genre'] = df['Genre'].apply(lambda x: x[1:-1].split(','))
df = df.explode('Genre')
df['Genre'].fillna(df['Genre'].mode()[0], inplace=True)
df['Genre'] = df['Genre'].apply(lambda x: x.lstrip().rstrip())
df.head()

,IMDb Rating,Runtime,Genre,Director,Cast,isHindi,Year
4,5.2,142,'Drama','Sanjay Leela Bhansali','Ranbir Kapoor',1.0,2007
4,5.2,142,'Musical','Sanjay Leela Bhansali','Ranbir Kapoor',1.0,2007
4,5.2,142,'Romance','Sanjay Leela Bhansali','Ranbir Kapoor',1.0,2007
5,7.1,141,'Comedy','Amit Joshi','Shahid Kapoor',1.0,2024
5,7.1,141,'Drama','Amit Joshi','Shahid Kapoor',1.0,2024


## Feature Engineering

In [55]:
genre_mean_rating = df.groupby('Genre')['IMDb Rating'].transform('mean')
df['Genre_mean_rating'] = genre_mean_rating
df.drop(['Genre'], axis=1, inplace=True)

director_mean_rating = df.groupby('Director')['IMDb Rating'].transform('mean')
df['Director_mean_rating'] = director_mean_rating
df.drop(['Director'], axis=1, inplace=True)

cast_mean_rating = df.groupby('Cast')['IMDb Rating'].transform('mean')
df['Cast_mean_rating'] = cast_mean_rating
df.drop(['Cast'], axis=1, inplace=True)

In [56]:
# for i in range(len(df)):
#     j = df.loc[i, 'Language']
#     j = j[1:-1].split(',')
#     flg = False
#     for k in j:
#         if k.strip()=="'Hindi'":
#             df.loc[i, "isHindi"] = 1
#             flg = True
#             break
#     if not flg:
#         df.loc[i, "isHindi"] = 0
# df.drop(['Language'], axis=1, inplace=True)

In [57]:
df.head()

,IMDb Rating,Runtime,isHindi,Year,Genre_mean_rating,Director_mean_rating,Cast_mean_rating
4,5.2,142,1.0,2007,6.546046,7.081481,6.339130
4,5.2,142,1.0,2007,6.481957,7.081481,6.339130
4,5.2,142,1.0,2007,6.309080,7.081481,6.339130
5,7.1,141,1.0,2024,6.258643,7.100000,5.895714
5,7.1,141,1.0,2024,6.546046,7.100000,5.895714


### Scaling the data so easily usable by SVM and logistic regression

In [58]:
#Scale the numerical columns
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['Runtime', 'isHindi', 'Year', 'Genre_mean_rating', 'Director_mean_rating', 'Cast_mean_rating']] = scaler.fit_transform(df[['Runtime', 'isHindi', 'Year', 'Genre_mean_rating', 'Director_mean_rating', 'Cast_mean_rating']])
df.head()


,IMDb Rating,Runtime,isHindi,Year,Genre_mean_rating,Director_mean_rating,Cast_mean_rating
4,5.2,0.401554,0.985711,-0.470917,0.628138,0.608887,-0.033658
4,5.2,0.401554,0.985711,-0.470917,0.394314,0.608887,-0.033658
4,5.2,0.401554,0.985711,-0.470917,-0.236422,0.608887,-0.033658
5,7.1,0.384253,0.985711,0.802478,-0.420438,0.624823,-0.463146
5,7.1,0.384253,0.985711,0.802478,0.628138,0.624823,-0.463146


# Training and Testing

### Defining the target and making a train test split

In [59]:
df['target'] = df['IMDb Rating'].apply(lambda x: 0 if x<3 else (1 if x<5 else (2 if x<7 else 3)))
df.drop('IMDb Rating', axis=1, inplace=True)

In [60]:
X = df.drop(['target'], axis=1)
y = df['target']

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Logistic Regression

In [62]:
from models.LogisticRegression import LogisticRegression
regr = LogisticRegression(regularization="l2")
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
accuracy_score(y_test, y_pred)

0.6129685916919959

### Support Vector Machine

In [63]:


from models.SVM import MulticlassSVM

n_classes = len(np.unique(y_train.to_numpy()))  # Number of classes

# Initialize and train multiclass SVM
svm = MulticlassSVM(n_classes,n_jobs=-1)  # Parallelize using all available CPU cores
svm.fit(X_train.to_numpy(), y_train.to_numpy())

y_pred = svm.predict(X_test.to_numpy())
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score for SVM:", accuracy)

Accuracy score for SVM: 0.6663289429246876


### Decision Tree Classifier

In [19]:
from models.DecisionTree import DTC
clf = DTC(40)
clf.fit(X_train.to_numpy(), y_train.to_numpy())
y_pred = clf.predict(X_test.to_numpy())
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score for Decision Tree:", accuracy)

Accuracy score for Decision Tree: 0.8959810874704491


### Random Forest Classifier

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score for Random Forest:", accuracy)

Accuracy score for Random Forest: 0.8824721377912867


### Gaussian Naive Bayes

In [21]:
from models.NaiveBayes import GNB
model = GNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Score for Gaussian Naive Bayes:", accuracy)

Accuracy Score for Gaussian Naive Bayes: 0.7149611617696724


### XGBoost Classifier

In [22]:
import xgboost as xgb
#Creating an XGBoost classifier
model = xgb.XGBClassifier()

#Training the model on the training data
model.fit(X_train, y_train)

#Making predictions on the test set
predictions = model.predict(X_test)

#Calculating accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy score for XGBoost:", accuracy)

Accuracy score for XGBoost: 0.8649105032083756
